In [2]:
!pip install --upgrade --force-reinstall pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 31.9 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
   

In [3]:
import pandas as pd
!pip install mysql-connector-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 35.2 MB/s eta 0:00:00


In [5]:
import requests
import pandas as pd
from sqlalchemy import create_engine

# Your N2YO API key and RDS credentials
api_key = 'DYBYU3-8PS72S-RYR93U-58XR'
rds_username = 'spaceboy'
rds_password = 'Flytweet04'
rds_hostname = 'spacex-project1.c9mbbltare10.us-east-1.rds.amazonaws.com'
rds_port = 3306
rds_dbname = 'sql_project'  # Make sure to put the actual name of your database here

# Satellite NORAD IDs from your screenshot
satellite_ids = [57150, 50174, 57416, 51778, 57733, 52377, 58453, 58092, 52691, 46083, 55483, 47604, 55949, 48147, 56423, 48442, 56779, 49418, 57229, 51463, 57637, 52116, 58236, 57914, 52554, 58673, 52997, 59018, 47648, 53430, 59393, 47351, 53666, 44942, 54002]

# Your observer position
observer_lat = '33.9703344'  # Removed parentheses as they are not needed
observer_long = '-118.4181400'
observer_alt = 0
seconds = 90

# Create a database connection
engine = create_engine(f'mysql+mysqlconnector://{rds_username}:{rds_password}@{rds_hostname}:{rds_port}/{rds_dbname}')

# Define a function to fetch and store satellite data
def fetch_and_store_satellite_data(sat_id):
    url = f"https://api.n2yo.com/rest/v1/satellite/positions/{sat_id}/{observer_lat}/{observer_long}/{observer_alt}/{seconds}/&apiKey={api_key}"
    response = requests.get(url)
    data = response.json()

    # Convert the positions data to a pandas DataFrame
    positions_df = pd.json_normalize(data['positions'])
    positions_df['satellite_id'] = sat_id  # Add the NORAD ID to the DataFrame

    # Insert the data into your MySQL database
    positions_df.to_sql('starlink_positions', con=engine, if_exists='append', index=False)

# Loop through all satellite IDs and fetch/store their data
for sat_id in satellite_ids:
    fetch_and_store_satellite_data(sat_id)
